# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые перешли на тарифы из проекта "Определение перспективного тарифа для телеком компании". Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Построим модель с максимально большим значением *accuracy*, по крайней мере 0.75.

## Оглавление:
* [1. Откроем и изучим файл](#1)
* [2. Разобьем данные на выборки](#2)
* [3. Исследуем модели](#3)
* [4. Проверим модель на тестовой выборке](#4)
* [5. Проверим модели на адекватность](#5)

## 1. Откроем и изучим файл <a class="anchor" id="1"></a>

Импортируем библиотеки.

In [1]:
# <импорт библиотеки pandas>
import pandas as pd

# <импорт библиотеки math>
import math

# <импорт библиотеки sklearn>
import sklearn

# <Отключение предупреждений>
import warnings
warnings.filterwarnings('ignore')

# <импорт библиотеки numpy>
import numpy as np

Прочитаем файл с данными.

In [2]:
# <чтение файлов с данными с сохранением в различные переменные>
df = pd.read_csv('datasets/users_behavior.csv')

Рассмотрим информацию по датафрейму и первые 5 строк:

In [3]:
# <рассмотрим датафрейм df>
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Пропуски отсутствуют.

Сменим тип данных на целочисленный, мне кажется в реальных задачах это было бы кстати, например для экономии памяти.

In [4]:
# <поменяем тип на целочисленный>
df['calls'] = df['calls'].astype('int64')
df['messages'] = df['messages'].astype('int64')

Оценим какой тариф предпочитает большинство пользователей, но вряд ли я смогу это использовать в проекте:

In [5]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Подробнее опишем значение каждого атрибута.

Таблица *df* (информация о поведении пользователя за месяц):

* *сalls* — количество звонков,
* *minutes* — суммарная длительность звонков в минутах,
* *messages* — количество sms-сообщений,
* *mb_used* — израсходованный интернет-трафик в Мб,
* *is_ultra* — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

### Вывод

* Данные не нуждаются в предобработке. 
* Пропуски отсутствуют. 
* Мы сменили тип данных, и я считаю это верное решение, так как сама суть понятий "количество звонков" и "количество сообщений" в том что они не дробятся. 
* Большинство пользователей предпочитают тариф "Ультра".

## 2. Разобьем данные на выборки <a class="anchor" id="2"></a>

Разобьем данные на три выборки: обучающую, валидационную и тестовую. Разобьем их в отношении 3:1:1. Пока нами не была изучена кросс-валидация, использую для этого метод *train_test_split()*:

In [6]:
# <Импортируем функцию из бибилиотеки sklearn>
from sklearn.model_selection import train_test_split

# <Поделим датафрейм на обучающую выборку и выборку, которую позже разделим на валидационную и тестовую.>
df_train, df_divide = train_test_split(df, test_size=0.40, random_state=12345)

# <Поделим df_divide валидационную и тестовую выборку.>
df_valid, df_test = train_test_split(df_divide, test_size=0.50, random_state=12345)

### Вывод

* Выборки разделены.

## 3. Исследуем модели <a class="anchor" id="3"></a>

Перед исследованием моделей машинного обучения разделим наши датафреймы на целевой признак который нужно предсказать и признаки, которые помогут нам его предсказать:

In [7]:
# <Разделим обучающий датафрейм на features и target - целевой признак>
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

# <Разделим валидационный датафрейм на features и target - целевой признак>
features_valid = df_valid.drop(['is_ultra'], axis=1)  
target_valid = df_valid['is_ultra']

Начнем с исследования модели дерева принятия решений.

In [8]:
# <Импортируем метод дерева принятия решений>
from sklearn.tree import DecisionTreeClassifier
# <Импортируем метод оценки доли правильных ответов>
from sklearn.metrics import accuracy_score

Оценим в цикле долю правильных ответов для разных глубин дерева принятия решений:

In [9]:
for depth in range(2,27,2):
    model = DecisionTreeClassifier(random_state=0, max_depth=depth, min_samples_split= 100)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print('max_depth =',depth,': {:.4f}'.format(accuracy))

max_depth = 2 : 0.7823
max_depth = 4 : 0.7854
max_depth = 6 : 0.7885
max_depth = 8 : 0.7854
max_depth = 10 : 0.7807
max_depth = 12 : 0.7807
max_depth = 14 : 0.7807
max_depth = 16 : 0.7807
max_depth = 18 : 0.7807
max_depth = 20 : 0.7807
max_depth = 22 : 0.7807
max_depth = 24 : 0.7807
max_depth = 26 : 0.7807


Лучшая модель с глубиной 6 с *accuracy* = 0.7885. *min_samples_split*  был подобран интуитивно, и улучшил результат на 6 тысячных по сравнению с дефолтным. Любые опыты с критерием и весом классов (*balanced*) ухудшали результат при *min_samples_split* = 100.

Перейдем к алгоритму случайного леса:

In [10]:
# <Импортируем алгоритм случайного леса>
from sklearn.ensemble import RandomForestClassifier

Оценим в цикле долю правильных ответов для разного количества оценщиков:

In [11]:
for estimators in range(2,16):
    model = RandomForestClassifier(random_state=8897, n_estimators=estimators, max_depth=6)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print('n_estimators =',estimators,': {:.4f}'.format(accuracy))

n_estimators = 2 : 0.7776
n_estimators = 3 : 0.7900
n_estimators = 4 : 0.7916
n_estimators = 5 : 0.7947
n_estimators = 6 : 0.7947
n_estimators = 7 : 0.7916
n_estimators = 8 : 0.7900
n_estimators = 9 : 0.7947
n_estimators = 10 : 0.8009
n_estimators = 11 : 0.7994
n_estimators = 12 : 0.8056
n_estimators = 13 : 0.8040
n_estimators = 14 : 0.8009
n_estimators = 15 : 0.8025


Лучшая модель с количеством оценщиков 12 и глубиной 6 с *accuracy* = 0.8056. *min_samples_split* лучше оставить дефолтным, он ухудшал результат.

Попробуем использовать *RandomizedSearchCV* для подбора параметров модели случайного леса, проверим модель выше и эту на тестовой выборке в следующем пункте. Я слышал что использование таких методов не рекомендуется в проекте без понимания того что в них происходит, поэтому постараюсь разъяснить что я понял, исследуя код:

* *param_grid* это словарь в котором указываются гиперпараметры и интервалы в которых они могут находится, или в случае если это категориальные признаки - возможные значения.
* функция *np.linspace()* последовательность чисел, количество - третий аргумент, сами числа от первого аргумента до второго. Если у нее два аргумента, выдает 50 чисел от первого аргумента до второго.
* в *np.arange* в качестве третьего аргумента шаг. Первый и второй аргумент - начало и конец последовательности. np.arange(0.5, 1, 0.1) - тут в последовательности не будет единицы но будет [0.5, 0.6, 0.7, 0.8, 0.9].
* *max_leaf_nodes* - максимальное количество листовых узлов (нижние узлы с ответами).
* *max_features* - количеcтво используемых признаков.
* *min_samples_split* - минимальное число объектов, при котором выполняется расщепление на узлы.
* *bootstrap* - Бутстрэп-агрегирование или бэггинг, алгоритм предназначенный для улучшения стабильности и точности алгоритмов машинного обучения, также уменьшает дисперсию и помогает избежать переобучения. Для меня - черный ящик.

In [12]:
# <Импортируем алгоритм случайного поиска по гиперпараметрам>
from sklearn.model_selection import RandomizedSearchCV

# <сетка гиперпараметров, по которой будет происходит случайный поиск>
param_grid = {
    'n_estimators': np.linspace(2, 100).astype(int),
    'max_depth': [None] + list(np.linspace(2, 10).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0, 5, 1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

# <случайный лес к которому будем подбирать параметры>
estimator = RandomForestClassifier(random_state = 8897)

# <модель>
rs = RandomizedSearchCV(estimator, param_grid, random_state=8897)

# <обучаем модель> 
rs.fit(features_train, target_train)

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs

In [13]:
# <набор параметров, которые подобрал случайный поиск.>
rs.best_params_

{'n_estimators': 38,
 'min_samples_split': 2,
 'max_leaf_nodes': 44,
 'max_features': 'auto',
 'max_depth': 8,
 'bootstrap': True}

In [14]:
# <предскажем целевой признак и подсчитаем долю правильных ответов.>
predictions = rs.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
accuracy

0.7993779160186625

Результат хуже, но тем не менее проверим эту модель на тестовой выборке.

Перейдем к логистической регрессии:

In [15]:
# <Импортируем метод логистической регрессии>
from sklearn.linear_model import LogisticRegression

In [16]:
# <Создадим модель лог. регрессии,>
model = LogisticRegression(random_state=12345)

# <обучим ее,>
model.fit(features_train, target_train)

# <предскажем целевой признак,>
predictions = model.predict(features_valid)

# <и подсчитаем долю правильных ответов.>
accuracy = accuracy_score(target_valid, predictions)

# <выведем долю правильных ответов>
accuracy

0.7589424572317263

Результат хуже чем у модели случайного леса или просто решающего дерева. Я осмотрел некоторые гиперпараметры типа *solver*, *class_weight*, *max_iter* и попробовал покрутить их, но они или давали тот же самый результат или еще сильней ухудшали его (использовал например советы отсюда https://overcoder.net/q/968297/%D0%BA%D0%B0%D0%BA-%D0%BF%D0%BE%D0%B2%D1%8B%D1%81%D0%B8%D1%82%D1%8C-%D1%82%D0%BE%D1%87%D0%BD%D0%BE%D1%81%D1%82%D1%8C-%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8-%D0%BB%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B9-%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D0%B8-%D0%B2-python-scikit).

### Вывод

* Мы разделили тренировочную выборку и валидационную на признаки(features) и целевой признак(target).
* Обучили дерево принятия решений и выбрали лучшие с максимальной глубиной 2 и 6.
* Обучили модель случайного леса, и выбрали лучшую с количеством оценщиков 12 и глубиной 6. Ее используем для проверки на тестовой выборке. Также используем модель, для которой подобрали параметры случайным поиском.
* Обучили модель логистической регресии и получили метрики хуже чем у предыдущих моделей.
* Открыли для себя несколько интересных методов, гиперпараметров, потренировались подбирать параметры.

## 4. Проверим модель на тестовой выборке <a class="anchor" id="4"></a>

Перед проверкой модели разделим наш тестовый датафрейм на features и target:

In [17]:
# <Разделим тестовый датафрейм на features и target - целевой признак>
features_test = df_test.drop(['is_ultra'], axis=1)  
target_test = df_test['is_ultra']

Проверим модель на тестовой выборке:

In [18]:
# <Создадим модель случайного леса,>
model = RandomForestClassifier(random_state=8897, n_estimators=12, max_depth=6)

# <обучим ее,>
model.fit(features_train, target_train)

# <предскажем целевой признак,>
predictions = model.predict(features_test)

# <и подсчитаем долю правильных ответов.>
accuracy = accuracy_score(target_test, predictions)

# <выведем долю правильных ответов>
print('accuracy =','{:.4f}'.format(accuracy))

accuracy = 0.7994


Проверим модель *rs* с гиперпараметрами, подобранными случайным поиском на тестовой выборке:

In [19]:
# <предскажем целевой признак,>
predictions = rs.predict(features_test)

# <и подсчитаем долю правильных ответов.>
accuracy = accuracy_score(target_test, predictions)

# <выведем долю правильных ответов>
print('accuracy =','{:.4f}'.format(accuracy))

accuracy = 0.8056


Модель, для которой мы подбирали гиперпараметры случайным поиском показала чуть-чуть лучший результат, чем та у которой мы подобрали только глубину и количество оценщиков. Тем не менее обе модели справились с поставленной задачей.

### Вывод

* Мы проверили две лучшие подобранные нами модели на тестовой выборке и получили устраивающий нас результат.
* Следующий шаг: научиться пользоваться кросс-валидацией и GridSearch для повышения accuracy в подобных задачах.

## 5. (бонус) Проверьте модели на адекватность <a class="anchor" id="5"></a>

Для проверки адекватности нашей модели напишем модель (функцию), предсказывающую значение признака *is_ultra* самым простым и тупорылым способом, 50 на 50, и сравним, лучше ли наша модель:

In [20]:
import random

random_predictions = np.random.randint(low = 0, high = 2, size = 643) 

# <подсчитаем долю правильных ответов.>
accuracy = accuracy_score(target_test, random_predictions)

# <выведем долю правильных ответов>
print('accuracy =','{:.4f}'.format(accuracy))

accuracy = 0.4961


Как видим случайные ответы ошибаются примерно в 50% случаев. Наша модель ошибается реже - только лишь в 19.44% случаев.

### Вывод

* Наша модель лучше, чем подкидывание монетки. Она прошла проверку на адекватность/вменяемость.